In [16]:
import pandas as pd 
import numpy as np 
import os

census = pd.read_csv('CENSUS_DATA.csv', sep=',', skiprows=13, header=1)
census.columns
gpt_data = pd.read_csv('gpt_comma_survey.csv')




In [ ]:
PATH = "CENSUS_DATA.csv"

# Load and skip metadata
df = pd.read_csv(PATH, skiprows=14, engine="python")

group_col = "Group"
edu_col   = "Educational attainment"

# find partition header rows
g = df[group_col].astype(str).str.strip()
edu = df[edu_col].astype(str).str.strip()

is_header = g.str.startswith("->") & (df[edu_col].isna() | (edu == "") | (edu.str.lower() == "nan"))

# depth = how many '->' markers in the Group string
depth = g.str.count(r"->")

#find contiguous runs of header rows 
#run_id increments at the START of each header run, and is 0 for non-header rows
start_of_header_run = is_header & ~is_header.shift(fill_value=False)
run_id = start_of_header_run.cumsum()
run_id = run_id.where(is_header, 0)

# within each header run, keep ONLY the deepest header row 
max_depth_in_run = depth.where(is_header).groupby(run_id).transform("max")

# If multiple rows tie for max depth, keep the LAST one in that run
is_deepest = is_header & (depth == max_depth_in_run)
is_last_of_deepest = is_deepest & (is_deepest.groupby(run_id).cummax()) & (
    is_deepest.groupby(run_id).cumcount(ascending=True)
    == is_deepest.groupby(run_id).transform(lambda s: np.flatnonzero(s.values)[-1] if s.any() else -1)
)

# tie-break: keep deepest + last by index among deepest
idx_last_deepest = (
    df.index.to_series()
      .where(is_deepest)
      .groupby(run_id)
      .transform("max")
)
keep_header = is_header & (df.index == idx_last_deepest)

#final keep mask: keep all non-header rows + only deepest headers 
keep = (~is_header) | keep_header

df_filtered = df.loc[keep].reset_index(drop=True)

# show dropped header rows
dropped_headers = df.loc[is_header & ~keep_header, [group_col, edu_col]]
print("Dropped header totals:", len(dropped_headers))

#save
df_filtered.to_csv("CENSUS_DATA_filtered_lowest_partition.csv", index=False)
print("Wrote CENSUS_DATA_filtered_lowest_partition.csv")


Dropped header totals: 45
Wrote CENSUS_DATA_filtered_lowest_partition.csv


In [ ]:
import pandas as pd
import numpy as np

RAW_PATH = "CENSUS_DATA.csv"

GROUP_COL = "Group"
EDU_COL   = "Educational attainment"

#load raw file (header starts at the row with "Group", so skip first 14 lines)
raw = pd.read_csv(RAW_PATH, skiprows=14, engine="python")

g = raw[GROUP_COL].astype(str).str.strip()
edu = raw[EDU_COL].astype(str).str.strip()

#header label rows: "-> ..." and blank education cell
is_header = g.str.startswith("->") & (raw[EDU_COL].isna() | (edu == "") | (edu.str.lower() == "nan"))

#depth = number of arrows in label
depth = g.str.count(r"->")
#find contiguous header "stacks" and keep only deepest one per stack
#    Example stack:
#     -> Division
#     -> Division -> 18-29
#     -> Division -> 18-29 -> Male   <-- keep this only
start_of_stack = is_header & ~is_header.shift(fill_value=False)
stack_id = start_of_stack.cumsum()
stack_id = stack_id.where(is_header, 0)

max_depth = depth.where(is_header).groupby(stack_id).transform("max")
is_deepest = is_header & (depth == max_depth)

# if ties exist, keep the last deepest row in that stack
idx_keep = (
    raw.index.to_series()
       .where(is_deepest)
       .groupby(stack_id)
       .transform("max")
)
keep_header = is_header & (raw.index == idx_keep)

#parse region/age/sex from deepest header labels and forward-fill down
raw["census_region"] = pd.Series([pd.NA] * len(raw), dtype="string")
raw["age_group"]     = pd.Series([pd.NA] * len(raw), dtype="string")
raw["sex"]           = pd.Series([pd.NA] * len(raw), dtype="string")

def parse_leaf_label(s: str):
    #split on -> , drop empties/spaces
    parts = [p.strip() for p in str(s).split("->") if p.strip()]
    # leaf label should end with [Region/Division, Age, Sex]
    if len(parts) >= 3:
        #fix age group: if last 2nd part is '61', change to '60+' (or if any age group is '61')
        age = parts[-2]
        if age == '61':
            age = '> 60'
        return parts[-3], age, parts[-1]
    return (pd.NA, pd.NA, pd.NA)

parsed = raw.loc[keep_header, GROUP_COL].apply(parse_leaf_label)

raw.loc[keep_header, "census_region"] = [t[0] for t in parsed]
raw.loc[keep_header, "age_group"]     = [t[1] for t in parsed]
raw.loc[keep_header, "sex"]           = [t[2] for t in parsed]

#forward-fill labels into the unlabeled rows below each header
raw[["census_region", "age_group", "sex"]] = raw[["census_region", "age_group", "sex"]].ffill()

#keep only non-header rows that are actually under a leaf header
leaf = raw.loc[~is_header].copy()
leaf = leaf.dropna(subset=["census_region", "age_group", "sex"])

#drop education "Total" rows (not a category)
leaf = leaf[leaf[EDU_COL].astype(str).str.strip().ne("Total")].copy()

#collapse educational attainment into 5 categories
def collapse_education(x: str) -> str:
    x = str(x).strip()

    # Graduate degree
    if x in {"Master's degree",
             "Professional degree beyond a bachelor's degree",
             "Doctorate degree"}:
        return "Graduate degree"

    # Bachelor's
    if x == "Bachelor's degree":
        return "Bachelor degree"

    # Some college / Associate
    if x in {"Some college, but less than 1 year",
             "1 or more years of college credit, no degree",
             "Associate's degree"}:
        return "Some college or Associate degree"

    # High school credential
    if x in {"Regular high school diploma",
             "GED or alternative credential"}:
        return "High school degree"

    #less than HS (everything below HS)
    #includes: N/A (less than 3 years old), no schooling, preschool, K, Grades 1-11, 12th no diploma
    return "Less than high school degree"

leaf["education_5"] = leaf[EDU_COL].apply(collapse_education)
#ID income-bin columns + melt to long pop_count
def is_bin_col(c):
    c = str(c).strip()
    if c.endswith("+"):
        c = c[:-1]
    if "-" in c:
        a, b = c.split("-", 1)
        return a.isdigit() and b.isdigit()
    return c.isdigit()

bin_cols = [c for c in leaf.columns if is_bin_col(c)]

#numeric conversion (remove commas)
for c in bin_cols:
    leaf[c] = (
        leaf[c].astype(str)
               .str.replace(",", "", regex=False)
               .replace("nan", np.nan)
    )
    leaf[c] = pd.to_numeric(leaf[c], errors="coerce").fillna(0).astype(np.int64)

long = leaf.melt(
    id_vars=["census_region", "age_group", "sex", "education_5"],
    value_vars=bin_cols,
    var_name="income_bin",
    value_name="pop_count",
)

# Clean
cat_cols = ["census_region", "age_group", "sex", "education_5", "income_bin"]
for c in cat_cols:
    long[c] = long[c].astype(str).str.strip()

#Full Cartesian product for poststrat bins: one row per combo (missing combos = 0)
levels = {c: sorted(long[c].unique()) for c in cat_cols}

full_index = pd.MultiIndex.from_product([levels[c] for c in cat_cols], names=cat_cols)

post_strat = (
    long.groupby(cat_cols, dropna=False)["pop_count"].sum()
        .reindex(full_index, fill_value=0)
        .reset_index()
)

#check and save
expected = int(np.prod([len(levels[c]) for c in cat_cols]))
actual = len(post_strat)
dups = post_strat.duplicated(subset=cat_cols).sum()

print("Any missing region/age/sex in long?", long[["census_region","age_group","sex"]].isna().any().any())
print("Expected rows (cartesian):", expected)
print("Actual rows:", actual)
print("Duplicate key rows:", dups)

OUT = "post_strat_long_full_cartesian_FIXED_v2.csv"
post_strat.to_csv(OUT, index=False)
print("Wrote:", OUT)


Any missing region/age/sex in long? False
Expected rows (cartesian): 1800
Actual rows: 1800
Duplicate key rows: 0
Wrote: post_strat_long_full_cartesian_FIXED_v2.csv
